In [1]:
import os
import random

import telebot
from telebot.types import KeyboardButton, ReplyKeyboardMarkup

In [ ]:
API_TOKEN = os.getenv("TG_BOT_TOKEN")
bot = telebot.TeleBot(API_TOKEN)

blackjack_games = {}  # For blackjack
user_states = {}  # For guess the number game
suggested_numbers = {}


# MENU


def get_game_menu():
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.row(
        KeyboardButton("Play Blackjack"), KeyboardButton("Play Guess the number")
    )
    return markup


@bot.message_handler(commands=["start", "menu"])
def show_game_menu(message):
    chat_id = message.chat.id
    bot.send_message(chat_id, "Choose a game:", reply_markup=get_game_menu())


# GUESS_THE_NUMBER


def get_keyboard():
    markup = ReplyKeyboardMarkup(resize_keyboard=True)
    markup.row(*(KeyboardButton(str(i)) for i in range(1, 4)))
    markup.row(*(KeyboardButton(str(i)) for i in range(4, 7)))
    markup.row(*(KeyboardButton(str(i)) for i in range(7, 10)))
    markup.row(KeyboardButton("0"), KeyboardButton("Delete"), KeyboardButton("Send"))
    return markup


def start_guess_game(chat_id):
    user_states[chat_id] = {"number": "", "last_msg_id": None}
    suggested_numbers[chat_id] = random.randint(0, 100)
    bot.send_message(
        chat_id,
        "Guess a number from 0 to 100",
        reply_markup=get_keyboard(),
    )


def delete_previous_message(chat_id):
    state = user_states.get(chat_id)
    if state and state["last_msg_id"]:
        try:
            bot.delete_message(chat_id, state["last_msg_id"])
        except Exception as e:
            print(f"Could not delete message: {e}")


def send_current_number(chat_id):
    state = user_states[chat_id]
    msg = bot.send_message(chat_id, f"Current: {state['number']}")
    state["last_msg_id"] = msg.message_id


# BLACKJACK


def deal_card():
    cards = [2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 11]
    return random.choice(cards)


def calculate_score(hand):
    score = sum(hand)
    while score > 21 and 11 in hand:
        hand[hand.index(11)] = 1
        score = sum(hand)
    return score


def start_blackjack():
    return {
        "player": [deal_card(), deal_card()],
        "dealer": [deal_card(), deal_card()],
        "game_over": False,
    }


def hand_text(hand):
    return ", ".join(map(str, hand))


def blackjack_menu(chat_id):
    game = blackjack_games[chat_id]
    bot.send_message(
        chat_id,
        f"Your hand: {hand_text(game['player'])} (Total: {calculate_score(game['player'])})\n"
        f"Dealer shows: {game['dealer'][0]}\nType /hit or /stand",
    )


@bot.message_handler(commands=["hit"])
def hit(message):
    chat_id = message.chat.id
    if chat_id not in blackjack_games:
        bot.send_message(chat_id, "Start a blackjack game with /blackjack")
        return
    game = blackjack_games[chat_id]
    if game["game_over"]:
        bot.send_message(chat_id, "Game over. Start a new one with /blackjack")
        return

    game["player"].append(deal_card())
    score = calculate_score(game["player"])

    if score > 21:
        game["game_over"] = True
        bot.send_message(
            chat_id,
            f"You busted with {hand_text(game['player'])} (Total: {score}). Dealer wins!",
        )
    else:
        blackjack_menu(chat_id)


@bot.message_handler(commands=["stand"])
def stand(message):
    chat_id = message.chat.id
    if chat_id not in blackjack_games:
        bot.send_message(chat_id, "Start a blackjack game with /blackjack")
        return

    game = blackjack_games[chat_id]
    game["game_over"] = True

    player_score = calculate_score(game["player"])
    dealer_score = calculate_score(game["dealer"])

    while dealer_score < 17:
        game["dealer"].append(deal_card())
        dealer_score = calculate_score(game["dealer"])

    result = (
        f"Your hand: {hand_text(game['player'])} (Total: {player_score})\n"
        f"Dealer hand: {hand_text(game['dealer'])} (Total: {dealer_score})\n"
    )

    if dealer_score > 21 or player_score > dealer_score:
        result += "You win!"
    elif dealer_score == player_score:
        result += "It's a tie"
    else:
        result += "You died"

    bot.send_message(chat_id, result)


# BUTTONNS


@bot.message_handler(func=lambda msg: True)
def game_selector_handler(message):
    chat_id = message.chat.id
    text = message.text

    if text == "Play Guess the number":
        start_guess_game(chat_id)
        return

    if text == "Play Blackjack":
        blackjack_games[chat_id] = start_blackjack()
        blackjack_menu(chat_id)
        return

    if chat_id in user_states and chat_id in suggested_numbers:
        target = suggested_numbers[chat_id]
        state = user_states[chat_id]

        if text.isdigit():
            state["number"] += text
        elif text == "Delete":
            state["number"] = state["number"][:-1]
        elif text == "Send":
            if state["number"]:
                number = int(state["number"])
                if number < target:
                    bot.send_message(
                        chat_id,
                        "The number I've picked is higher.",
                        reply_markup=get_keyboard(),
                    )
                elif number > target:
                    bot.send_message(
                        chat_id,
                        "The number I've picked is lower.",
                        reply_markup=get_keyboard(),
                    )
                else:
                    bot.send_message(chat_id, f"Correct! It was {target}.")
                    del suggested_numbers[chat_id]
                    del user_states[chat_id]
                    return
                state["number"] = ""
            else:
                bot.send_message(chat_id, "You haven't entered a number yet.")
        delete_previous_message(chat_id)
        send_current_number(chat_id)


bot.polling()